# PSTAT 234 Homework 3 
__Author: Laura Urbisci__  
__Due date: May 13, 2018 by 10 pm__


## Preparing the data
Before I could do the problems assigned in the homework, I needed to prepare the data using a pickle object.

In [ ]:
# Import modules
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import subprocess as sp
import pickle # to serialize/unserialize python data objects

import helper_basketball as h
import imp
imp.reload(h);

In [ ]:
# This is our original data set:
allshots = pickle.load(open('allshots2016-17.pkl', 'rb'))
allmade = allshots
allmade[1:10]

In [ ]:
## bin edge definitions in inches
xedges = (np.linspace(start=-25, stop=25, num=151, dtype=np.float)) * 12
yedges = (np.linspace(start= -4, stop=31, num=106, dtype=np.float)) * 12

## 2d histogram containers for binned counts and smoothed binned counts
all_counts = {}
all_smooth = {}

## data matrix: players (row) by vectorized 2-d court locations (column)
for i, one in enumerate(allmade.groupby('PlayerID')):
    
    ## what does this line do?
    pid, pdf = one
    
    ## h.bin_shots: what is this function doing?
    tmp1, xedges, yedges = h.bin_shots(pdf, bin_edges=(xedges, yedges), density=True, sigma=2)
    tmp2, xedges, yedges = h.bin_shots(pdf, bin_edges=(xedges, yedges), density=False)
    
    ## vectorize and store into dictionary
    # see: https://stackoverflow.com/questions/18691084/what-does-1-mean-in-numpy-reshape
    all_smooth[pid] = tmp1.reshape(-1)
    all_counts[pid] = tmp2.reshape(-1)

In [ ]:
pickle.dump(np.stack(all_smooth.values()).T, open('allpatterns2016-17.pkl', 'wb'))

In [ ]:
X = pickle.load(open('allpatterns2016-17.pkl', 'rb'))

## Problem 1: 

__PSTAT 134 and 234__: Experiment with different number of `n_components` to change the number of bases vectors. Visualize the bases vectors.

What value of $r$ seem to be too small? (`r` is too small to represent diversity of shooting modes) What value of $r$ seem to be too large? (`r` is too large and some bases seem to be duplicated). Note that, if a basis were a perfect duplicate of another (they will not be, but could be similar), you would use one basis instead of two.

__PSTAT 234 (optional for 134)__: Choose two different choices for number of components, say $r_1=3$ and $r_2=20$. Reconstruct the shooting pattern of at least two player using 3 bases and 20 bases. Is there any difference between the reconstruction?

- For a given player, plot the original shooting frequencies and corresponding reconstruction for $r \in \{3,20\}$.

Compute the difference: i.e., the norm of the difference  $ \min_{W_r,H_r} \| X - W_rH_r \|_F$. Plot the approximation error as a function of $r$. (Note the subscript $r$ makes the choice of $r$ explicit.) Choose at least 10 different choices of $r$. Based on this plot, what can you say about choosing $r$?

In [ ]:
## Non-negative Matrix Factorization
def non_negative_marix_decomp(n_components,train_data):
    import sklearn.decomposition as skld
    model = skld.NMF(n_components=n_components, init='nndsvda', max_iter=500, random_state=0)
    W = model.fit_transform(train_data)
    H = model.components_
    nmf = (W,H)
    return(nmf)

In [ ]:
#X = np.stack(all_smooth.values()).T 
X.shape # Dim is currently row is (x_loc*y_loc) and column is now the number of players

### Problem 1a)  $r_1 = 3$

In [ ]:
r = 3
W_3,H_3 = non_negative_marix_decomp(n_components = r,train_data = X)
X3_hat = np.matmul(W_3,H_3) 

### Problem 1b)  $r_2 = 20$

In [ ]:
r = 20
W_20,H_20 = non_negative_marix_decomp(n_components = r,train_data = X)
X20_hat = np.matmul(W_20,H_20) 

### Problem 1c)  Visualize shooting pattern for two different players. Is there any difference between the reconstructions?

In this scenario, we find some cases that the number of bases ($r$) are too small or too big. Our ultimate goal in these situations is to try to project the original data in a dimension that is lower than what the original data was in and utlize the optimal number of bases which depicts the different aspects in players' shooting patterns. 


From our two sets of figures, we see that a basis of $r=20$ more accurately describes the shooting pattern than $r=3$. We see more of the lighter areas found in the original shooting pattern plots show up in the estimated plots. I think a basis of 3 is too small for our situation.

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(20,60))

# I took the 34th player appearing in first column 
h.plot_shotchart(X[:,33], xedges, yedges, ax=ax[0]) 
h.plot_shotchart(X3_hat[:,33], xedges, yedges, ax=ax[1])
h.plot_shotchart(X20_hat[:,33], xedges, yedges, ax=ax[2])
ax[0].set_title('Original Shooting Pattern for 34th player in data set')
ax[1].set_title('Estimated Shooting Pattern (r=3) for 34th player in data set')
ax[2].set_title('Estimated Shooting Pattern (r=20) for 34th player in data set')

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(20,60))

# I took the last player appearing in first column 
h.plot_shotchart(X[:,361], xedges, yedges, ax=ax[0]) 
h.plot_shotchart(X3_hat[:,361], xedges, yedges, ax=ax[1])
h.plot_shotchart(X20_hat[:,361], xedges, yedges, ax=ax[2])
ax[0].set_title('Original Shooting Pattern for last player in data set')
ax[1].set_title('Estimated Shooting Pattern (r=3) for last player in data set')
ax[2].set_title('Estimated Shooting Pattern (r=20) for last player in data set')

### Problem 1d) Compute the Frobenious norm between the actual and estimated shooting patterns for several values of number of bases, $r = 1,\ldots,10$.

From the plot we can see that as r increases, the Frobenious norm aka our approximation error decreases. It would appear that a higher number is preferable. Going back to our original comparison of $r=3$ and $r=20$, we once again see that $r=20$ is the preferrable basis, since it has a lower approximation error.  

In [ ]:
# Frobenious norm
from numpy import linalg as LA 

# To compute Frobenious norm for r=3
LA.norm(X-np.matmul(W_3,H_3),'fro')

In [ ]:
# To compute Frobenious norm for r=20
LA.norm(X-np.matmul(W_20,H_20),'fro') # this is lower than r=3

In [ ]:
from numpy import linalg as LA # To compute Frobenious norm
r_values = np.arange(10)+1
error_norm = []
for r in r_values:
    W,H = non_negative_marix_decomp(n_components = r,train_data = X)
    error_norm.append(LA.norm(X-np.matmul(W,H),'fro'))

In [ ]:
plt.plot(r_values,error_norm)

## Problem 2

__PSTAT 134 and 234__: In the previous question, NMF gave us a set of bases to describe each player. So, the comparison is through a standard set of shooting styles. We may also approach the comparison more directly.

* In this problem, we compare of players' shooting styles to each other directly. What we are interested in is pairwise correlation between shooting patterns. Let $X_i$ represent the column in the smoothed shooting pattern for player $i$. Then, we want to compute   
    $$ R = [\text{Cor} (X_i, X_j)]_{i,j} $$ for all player combinations $i,j\in\{1,2,\dots,362\}$. What is the correct orientation of matrix $X$? What should be the dimension of matrix $R$?   
    _Note: if your command is not running properly, you may be running into the issue of using too much memory, and your notebook session is rebooted by the server as a result._
    
* Visualize matrix $R$ with [seaborn.heatmap](https://seaborn.pydata.org/generated/seaborn.heatmap.html) function.

* Identify 2 pairs of players with highest similarities (positive correlation) and 2 pairs with lowest similarity (negative correlation). Plot their shooting pattern. What do you observe?

__PSTAT 234 (optional for 134)__: Perform hierarchical clustering with matrix $R$, and visualize the clustered matrix.

### Problem 2a) Correlation matrix

We do not need to transpose matrix X to calculate the R matrix. It's orientation is currently correct. When we use np.corrcoef() we just need to specify rowvar=0 because each coordinate is then represented by rows and the columns represent the variable (basketball player). The matrix R has pairwise correlation for each player. Therefore, it should have a dimension of (362, 362) since there are 362 basketball players. 

In [ ]:
?np.corrcoef # go to help to determine that we need to add rowvar=0

In [ ]:
R = np.corrcoef(X,rowvar=0) # Transpose to compute correlation between columns

In [ ]:
R.shape

### Problem 2b) Heatmap

Looking at the heatmap it's hard to detect which players have the highest and lowest similarities. So in the next section, I'll plot a clustered matrix

In [ ]:
import seaborn as sns; sns.set()
ax = sns.heatmap(R)

### Problem 2c) Hierarchical clustering

Now rearranging the data makes it easier to see which players are similiar and dissimilar. The darker the color, the higher the similarity. Using the information found in np.argsort(cluser_id) I can find 2 sets of players that are similiar and 2 sets of players that are dissimilar. When I plot their shooting patterns, we see that the players that had high correlation had similar shooting patterns and players that had low correlation had very different shooting patterns.

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist

In [ ]:
Z = linkage(X.T,method='complete',metric='correlation') # transpose X to cluster players not observations

In [ ]:
plt.title('Hierarchical Clustering Dendrogram (truncated)')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
)
plt.show()

In [ ]:
# obtaining the clustering id for each player

from scipy.cluster.hierarchy import fcluster
no_max_clust = 10
cluster_id = fcluster(Z,no_max_clust,criterion='maxclust') 
cluser_id # first player belongs in 7th cluster
np.sort(cluster_id) # now gets index associated with arrangment ids
np.argsort(cluster_id) # rearrange players by correlation pattern

In [ ]:
R_clust = np.corrcoef(X.T[np.argsort(cluster_id)])

In [ ]:
ax = sns.heatmap(R_clust)

In [ ]:
# high similarities

fig, ax = plt.subplots(1,2, figsize=(20,60))

# I took the 34th player appearing in first column 
h.plot_shotchart(X[:,199-1], xedges, yedges, ax=ax[0]) 
h.plot_shotchart(X[:,354-1], xedges, yedges, ax=ax[1]) 
ax[0].set_title('Original Shooting Pattern for 199th player in data set')
ax[1].set_title('Original Shooting Pattern for 354th player in data set')

In [ ]:
# low similarities

fig, ax = plt.subplots(1,2, figsize=(20,60))

# I took the 34th player appearing in first column 
h.plot_shotchart(X[:,168-1], xedges, yedges, ax=ax[0]) 
h.plot_shotchart(X[:,8-1], xedges, yedges, ax=ax[1]) 
ax[0].set_title('Original Shooting Pattern for 121th player in data set')
ax[1].set_title('Original Shooting Pattern for 126th player in data set')

## Problem 3

__PSTAT 134 and 234__: How would you use the coefficients matrix $H$ from NMF  or the correlation matrix $R$ (computed above) to differentiate between types of players? Consider what the coefficients represent, and how you can use them to discriminate player types.

Give your thought process, reasoning for your chosen method, and the results. Do they look reasonable? Do you expect any of the comparison to be similar to any of the [figures here](https://fastbreakdata.com/classifying-the-modern-nba-player-with-machine-learning-539da03bb824)? Why, or why not? Can you verify your intuition?

### Problem 3a)
The correlation matrix $R$ gives the correlation between each player, while the coefficient matrix $H$ depicts the importance due to the shooting aspect for each player. I would use the $H$ matrix to characterize the shooting style of each player, while I would use the $R$ matrix to see which players had similar shooting styles. Since my intention is to compare players, it makes more sense to use the $R$ matrix. The coefficients of the $H$ matrix tell you the most common basis function of a player, but it is otherwise hard to use this information to interpret one player's shooting style in regards another. 

### Problem 3b)
If I wanted to discriminate player types, I would use the hierarchical clustering which uses correlation to group the players. From there, I would pick a distance and use the clusters at that level to discriminate player types (See the plot of the Hierarchical Clustering Dendrogram (truncated)). The website used K-means clustering which could give similiar results to our hierarchical clustering. 

## Problem 4

__PSTAT 134 and 234__: Suppose you are in charge of a basketball team. How would you use this information? How would you use what you have learned from analyzing the data, and what other questions would you like to answer with further analysis.

First, I would use the information on different player types to help form my team's roster. If I was starting a team from scratch, I would want to make sure I have players that are skilled in different aspects to round out my team. Once I formed my team, I would use this data during games to checkout the opponent's players skills to form my defense stragety. Now I remember Prof Franks said it was difficult to analyze defense, but if I had information on player's defensive strengths and preferred styles, I would use that to inform my game time stragety as well. I would position players that would take advantage of the opponent's weaknesses.